## Variational Monte Carlo for the ground state of the Harmonic Oscillator

In [1]:
from matplotlib import use
use('nbAgg')

In [2]:
import numpy as np

### Service routines (statistical analisys)

In [3]:
def UnweightedAvg(meanList,errorList):
    mean=np.sum(meanList)/(len(meanList)+0.0)
    error=0.0;
    for e in errorList:
        error=error+e*e
    error=sqrt(error)/len(errorList)
    return (mean,error)


def WeightedAvg (means, errors):
    zeroErrors = False
    for i in errors:
        if i == 0.0:
            zeroErrors = True
    
    if (not zeroErrors):
        weights = map (lambda x: 1.0/(x*x), errors)
        norm = 1.0/np.sum(weights)
        weights = map(lambda x: x*norm, weights)
        avg = 0.0
        error2 = 0.0
        for i in range (0,len(means)):
            avg = avg + means[i]*weights[i]
            error2 = error2 + weights[i]**2*errors[i]*errors[i]
        return (avg, math.sqrt(error2))
    else:
        return (np.sum(means)/len(means), 0.0)


def MeanErrorString (mean, error):
     if (mean!=0.0):
          meanDigits = np.floor(np.log(np.abs(mean))/np.log(10))
     else:
          meanDigits=2
     if (error!=0.0):
          rightDigits = -np.floor(np.log(error)/np.log(10))+1
     else:
          rightDigits=2
     if (rightDigits < 0):
          rightDigits = 0
     formatstr = '%1.' + '%d' % rightDigits + 'f'
     meanstr  = formatstr % mean
     errorstr = formatstr % error
     return (meanstr, errorstr)


def c(i,x,mean,var):
    N=len(x)
    if var==0:#if the variance is 0 return an effectively infinity corr
        return 1e100
#    print (len(x([0:N-1])),len(x([i:N])))
    corr=1.0/var*1.0/(N-i)*np.sum((x[0:N-i]-mean)*(x[i:N]-mean))
    return corr
                         
def Stats(x):
    N=len(x)
    mean=np.sum(x)/N
    xSquared=(x-mean)*(x-mean)
    var=np.sum(xSquared)/N
    i=0          
    tempC=0.5
    kappa=0.0
    while (tempC>0 and i<(N-1)):
        kappa=kappa+2.0*tempC
        i=i+1
        tempC=c(i,x,mean,var) 
    if kappa == 0.0:
        kappa = 1.0
    Neff=(N+0.0)/(kappa+0.0)
    error=np.sqrt(var/Neff)
    return (mean,var,error,kappa)

### Service Class (visualizations)

In [4]:
from matplotlib import pyplot as plt

In [5]:
class Histogram:
        Hmin=0.0
        Hmax=5.0
        delta=0.05
        numSamples=0
        bins=np.array([0])
        def add(self,val):
                if self.min<val and val<self.max:
                        index=int((val-self.min)/self.delta)
                        self.bins[index]=self.bins[index]+1
                self.numSamples=self.numSamples+1
        def printMe(self):
                for i in range(0,len(self.bins)):
                        print (self.min+self.delta*i,self.bins[i]/(self.numSamples+0.0))

        def plotMe(self,fileName=""):
                print ("plotting")
                plt.figure(0)
                plt.clf()
                self.bins=self.bins/self.numSamples
                xCoord=np.array([self.min+self.delta*i for i in range(0,len(self.bins))])
                plt.plot(xCoord,self.bins)
                plt.gca().xaxis.major.formatter.set_scientific(False)
                if not(fileName==""):
                   plt.savefig(fileName)
                else:
                   plt.show()
        def plotMeNorm(self,fileName):
                print ("plotting")
                plt.figure(0)
                plt.clf()
                self.bins=self.bins/self.numSamples
                xCoord=np.array([self.min+self.delta*i for i in range(0,len(self.bins))])
                plt.plot(xCoord,self.bins/(xCoord*xCoord+0.0001))
                plt.gca().xaxis.major.formatter.set_scientific(False)
                plt.savefig(fileName)
                plt.show()

        def __init__(self,Hmin,Hmax,numBins):
                self.min=Hmin
                self.max=Hmax
                self.delta=(Hmax-Hmin)/(numBins+0.0)
                self.bins=np.zeros(numBins)+0.0
                numSamples=0

### Numerical Service Routines (Laplacian, ...)

In [6]:
def LaplacianPsiOverPsi(coords,WaveFunction):
    total=0.0
    delta=0.0001
    tempVal3=WaveFunction(coords)
    for i in range(0,len(coords)):
        for j in range(0,len(coords[0])):
            coords[i,j]=coords[i,j]+delta
            tempVal=WaveFunction(coords)
            coords[i,j]=coords[i,j]-2*delta
            tempVal2=WaveFunction(coords)
            coords[i,j]=coords[i,j]+delta
            total +=(tempVal+tempVal2)-2.0*tempVal3
    return total/(delta*delta*tempVal3)

def dist(x,y):
    return np.sqrt(np.dot(x-y,x-y))

def GetRandomUniformVec(sigma,vec):
    vec[0]=2.0*sigma*(np.random.random()-0.5)
    vec[1]=2.0*sigma*(np.random.random()-0.5)
    vec[2]=2.0*sigma*(np.random.random()-0.5)
#def GetRandomUniform(sigma,ndim):
    #return np.array([(np.random.random()-0.5)*2*sigma for _ in range(0,ndim)])

### Harmonic oscillator Class ( Trial Wavefunction and Potential)

In [7]:
class HOClass:
  def __init__(self):
    self.coord = np.zeros(1,np.float)
    self.SetParams([0.32])
    self.name = 'HO_WF'
  def SetParams(self,params):
     self.params=params.copy()
  def SetCoords(self,coords):
     self.coords=coords.copy()
  def WaveFunction(self,R):
     alpha=self.params[0]
     return         np.exp(-alpha*R**2)
     #return        R*exp(-alpha*R**2)
     #return        (R*R-0.5)*exp(-alpha*R**2)
     #return        (4.*alpha*R*R-1.)*exp(-0.5*R**2)
     #return        (R+alpha-0.5)*exp(-0.5*R**2)
  def LocalEnergy(self,R):
     KE=-0.5*LaplacianPsiOverPsi(R,self.WaveFunction)
     #change this to actually calculate V
     V=  0.5*R**2
     return V+KE

#### Test of the WaveFunction function

In [8]:
from numpy import zeros, exp
#from VMCHelp import *
#from VMC_HO import *

HO=HOClass()
HO.SetParams([0.33])
coords=zeros((1,1),float)
coords[0,0]=0.8
energyVal=HO.WaveFunction(coords)
print ("Your wavefunction is working if this is ", exp(-0.33*0.8**2),energyVal)

Your wavefunction is working if this is  0.8096121282623017 [[0.80961213]]


### Variational Monte Carlo function (single run)

In [9]:
def VMC(WF,numSteps):
    EnergyList=np.zeros(numSteps)
    myHist=Histogram(-5,5,500)
   #pseudoCode:
    delta=1.618034*2.
   #Looping over steps:
    movesAttempted=0.0
    movesAccepted=0.0
    R=np.zeros((1,1),float)
    R=WF.coords.copy()
    nR=np.zeros((1,1),float)
    Psi=WF.WaveFunction(R)
    energy=WF.LocalEnergy(R)
    for step in range(numSteps):
      #choose new coordinates
       dR = np.random.uniform(-delta,delta,1)
       nR = R + dR
      #evaluate the wave function on these new cooridinates
       newPsi=WF.WaveFunction(nR)
       if ( newPsi**2/Psi**2 > np.random.uniform() ):
         #accept the move making sure coords and oldWaveFunction has the up to date information
          R=nR.copy()
          Psi=newPsi
          movesAccepted+=1.
          energy=WF.LocalEnergy(R)
          EnergyList[step]=energy
       else:
        #reject the move making sure coords and oldWaveFunction has the up to date information
          EnergyList[step]=energy
       movesAttempted+=1.
       myHist.add(R[0,0])
    print ("Acceptance ratio: ", movesAccepted/movesAttempted   )
    myHist.plotMe("density-"+str(WF.params)+".pdf")
    return EnergyList


#### Test of the Local Energy and of the VMC single run

In [10]:
#from VMCHelp import *
#from VMC_H import *
#from stats import Stats 
from numpy import zeros, float, array
from matplotlib import pyplot as plt

HO=HOClass()
alpha=[0.33]
HO.SetParams(alpha)
R=zeros((1,1),float)
R[0,0]=0.5
HO.SetCoords(R)
local_energy=alpha[0] + ( 0.5 - 2.*alpha[0]**2 )*R[0,0]**2
print ("The Local Energy should be",local_energy,HO.LocalEnergy(R))
print ("The correct answer is approximately 0.543")
energyList=VMC(HO,100000)
print ("A long run gives ",Stats(array(energyList)))
fig=plt.figure(1, figsize=(7, 4), dpi=100)
s0 = fig.add_subplot(1, 1, 1)
s0.plot(energyList)
plt.savefig("EnergyGraph.pdf")
plt.show()

The Local Energy should be 0.40055 [[0.40054999]]
The correct answer is approximately 0.543
Acceptance ratio:  0.41714
plotting
A long run gives  (0.5449962536299392, 0.09194071007241335, 0.0019446754357542375, 4.11326228332083)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Optimize function (Direct search in the parameter space)

In [11]:
def Optimize(HO):
   optimizeList=[]
  #add things here to loop over alpha and do optimization
   bestAlpha=0.
   bestEnergy=0.
   oldene=100.
   for alpha in np.arange(0.15,.95,0.05) :
      HO.SetParams([alpha])
      energyList=VMC(HO,100000)
      energy, variance, error, corrtime = Stats(np.array(energyList))
      optimizeList.append([alpha,energy,error])
      print (" alpha= %g , energy= %g , err = %g , ac-time= %g" % (alpha,energy,error,corrtime))
      if energy<=oldene :
         bestAlpha=alpha
         bestEnergy=energy
         oldene=energy
   return (optimizeList,bestAlpha,bestEnergy)

### Search of Minimum with set of Monte Carlo runs

In [12]:
#from VMCHelp import *
#from VMC_H import *
#from stats import Stats
from numpy import random, zeros, array
from matplotlib import pyplot as plt 
 
#set up wave function as always

HO=HOClass()
HO.SetParams([0.8])
R=zeros((1,1),float)
R[0,0]=random.uniform(-1.5,1.5,1) 
print (R[0,0])
HO.SetCoords(R)
(optimizeList,bestAlpha,bestEnergy)=Optimize(HO)
print ("The optimum alpha is ",bestAlpha)
optimizeList=array(optimizeList)
fig=plt.figure(2, figsize=(9, 4), dpi=100)
s1 = fig.add_subplot(1, 2, 1)
s2 = fig.add_subplot(1, 2, 2)
s1.errorbar(optimizeList[:,0],optimizeList[:,1],optimizeList[:,2], marker='.', mec='red', mfc='red', elinewidth=2, ecolor='red',  mew=2)
s2.plot(optimizeList[:,0],optimizeList[:,2], marker='.', mec='red', mfc='red', mew=2)
#plt.axis([0., 1., bestEnergy-0.1, 1.3*bestEnergy])
plt.savefig("HO-optimize.png")
plt.show()

-1.1412257049885661
Acceptance ratio:  0.55538
plotting
 alpha= 0.15 , energy= 0.904268 , err = 0.00678732 , ac-time= 4.0436
Acceptance ratio:  0.50426
plotting
 alpha= 0.2 , energy= 0.724398 , err = 0.00465519 , ac-time= 3.85146
Acceptance ratio:  0.46527
plotting
 alpha= 0.25 , energy= 0.629424 , err = 0.00336917 , ac-time= 3.90039
Acceptance ratio:  0.43363
plotting
 alpha= 0.3 , energy= 0.564364 , err = 0.00229799 , ac-time= 3.81573
Acceptance ratio:  0.40734
plotting
 alpha= 0.35 , energy= 0.53179 , err = 0.00163836 , ac-time= 4.04216
Acceptance ratio:  0.38295
plotting
 alpha= 0.4 , energy= 0.512519 , err = 0.00106155 , ac-time= 4.42556
Acceptance ratio:  0.36283
plotting
 alpha= 0.45 , energy= 0.502273 , err = 0.000501968 , ac-time= 4.67176
Acceptance ratio:  0.34771
plotting
 alpha= 0.5 , energy= 0.5 , err = 5.16644e-11 , ac-time= 4.24607
Acceptance ratio:  0.32964
plotting
 alpha= 0.55 , energy= 0.502196 , err = 0.000487365 , ac-time= 5.20584
Acceptance ratio:  0.31631
plottin

<IPython.core.display.Javascript object>